In [2]:
import cv2
import math
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import os
import time



In [3]:
save_counter = 0
max_saves = 5 # 문제가 되는 구간 5장만 저장 할려고

# 이미지 저장을 위한 카운터 및 저장 디렉토리 설정
save_counter = 0
max_saves = 5  # 최대 저장할 이미지 수
save_directory = "/Users/2suyeon/Desktop/zerobase/미니프로젝트/DL_project/resultdata"  # 이미지를 저장할 경로

if not os.path.exists(save_directory):
    os.makedirs(save_directory)



In [4]:
def calculate_iou_and_remaining_area(iou_box, other_box):
    """
    두 박스 사이의 교집합 영역과 IoU 박스의 남은 넓이를 계산식
    """
    # 교집합 영역의 (x, y) 좌표
    xa = max(iou_box[0], other_box[0])
    ya = max(iou_box[1], other_box[1])
    xb = min(iou_box[2], other_box[2])
    yb = min(iou_box[3], other_box[3])

    # 겹치는 영역의 넓이
    intersection_area = max(0, xb - xa) * max(0, yb - ya)

    # IoU 박스의 넓이
    iou_box_area = (iou_box[2] - iou_box[0]) * (iou_box[3] - iou_box[1])
    
    # 남은 IoU 박스의 넓이
    remaining_iou_area = iou_box_area - intersection_area

    return intersection_area, remaining_iou_area

# 각도 계산 함수
def calculate_angle(x1, y1, x2, y2):
    angle_radians = math.atan2(y2 - y1, x2 - x1)
    angle_degrees = math.degrees(angle_radians)
    return angle_degrees



#remaining_iou_area 이 남은 넓이 임

model = YOLO("/Users/2suyeon/Desktop/zerobase/미니프로젝트/DL_project/best.pt")
cap = cv2.VideoCapture("/Users/2suyeon/Desktop/zerobase/미니프로젝트/DL_project/data/경전철_1.mp4")


w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
out = cv2.VideoWriter('/Users/2suyeon/Desktop/zerobase/미니프로젝트/DL_project/resultdata/visioneye-distance-calculation3.avi', cv2.VideoWriter_fourcc(*'MJPG'), fps, (w, h))



In [5]:
name_dict = {
  0: "bus",
  1: "fire hydrant",
  2: "car",
  3: "person",
  41: "person",
  4: "tree",
  5: "truck",
  6: "scooter",
  7: "bicycle",
  8: "bench",
  9: "bus_stop",
  10: "utility_pole",
  11: "motorcycle",
  12: "obstacle",
  13: "traffic light",
  14: "green",
  15: "elevator",
  16: "stair",
  17: "manhole",
  18: "dog",
  19: "trash",
  20: "red",
  21: "train_station"
}



In [6]:
#iou start

# iou_box 정의
iou_box = [int(w/4), int(h/4), int(3*w/4), int(3*h/4)]
iou_box_area = (iou_box[2] - iou_box[0]) * (iou_box[3] - iou_box[1])

# 위험 감지 관련 임계값 설정
iou_threshold = 0.8
max_detections = 5  # 최대 시각화할 이미지 수
detections = 0  # 현재까지 시각화한 이미지 수

#iou end

# #distance start


center_point = (w/2,h-40)

pixel_per_meter = 10

txt_color, txt_background, bbox_clr = ((0, 0, 0), (255, 255, 255), (255, 0, 255))

#distance end





while True:
    ret, im0 = cap.read()
    if not ret:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    
    results = model.track(im0, persist=True)
    boxes = results[0].boxes.xyxy.cpu().numpy()

    # track_ids를 안전하게 초기화
    track_ids = []
    if results[0].boxes.id is not None:
        track_ids = results[0].boxes.id.int().cpu().tolist()
    else:
        # 객체가 감지되지 않은 경우, track_ids는 빈 리스트로 남음
        track_ids = []

    
    

    detected_boxes = [(10, 20, 110, 220), (150, 160, 300, 460)]  # 예시 바운딩 박스

    #IoU 박스 그리기
    cv2.rectangle(im0, (iou_box[0], iou_box[1]), (iou_box[2], iou_box[3]), (255, 0, 0), 2)

    for box, class_id in zip(boxes, class_ids):
        class_name = name_dict.get(class_id, "Unknown")  # 클래스 ID에 해당하는 이름을 가져옴
        print(f"Detected object: {class_name} with bounding box: {box}")
    # 현재 바운딩 박스의 영역 계산
        current_box_area = (box[2] - box[0]) * (box[3] - box[1])
        intersection_area, remaining_iou_area = calculate_iou_and_remaining_area(iou_box, box)
    # remaining_iou_area이 남은 넓이임
    
    # 남은 IoU 영역이 임계값 이하인 경우 경고 및 이미지 저장
        if remaining_iou_area < iou_threshold * iou_box_area:
            # 경고 메시지 출력 또는 시각적 표시
            cv2.rectangle(im0, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 0, 255), 2)
            x1, y1 = int((box[0] + box[2]) / 2), int((box[1] + box[3]) / 2)
            
           


            cv2.putText(im0, f"DANGER. {class_name} is ahead.{class_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.waitKey(1)

            # 위험 상황 시 이미지 저장 
            if save_counter < max_saves:
                save_path = os.path.join(save_directory, f"danger_{save_counter}.jpg")
                cv2.imwrite(save_path, im0)
                print(f"Image saved to {save_path}")
                save_counter += 1
        
        
    cv2.imshow('Processed Frame', im0)

        # 'q' 키를 누르면 루프를 빠져나옵니다.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 7 persons, 7 obstacles, 70.2ms
Speed: 4.2ms preprocess, 70.2ms inference, 6.0ms postprocess per image at shape (1, 3, 384, 640)


NameError: name 'class_ids' is not defined

In [13]:
name =results[0]

name

ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'bus', 1: 'fire hydrant', 2: 'car', 3: 'person', 4: 'tree', 5: 'truck', 6: 'scooter', 7: 'bicycle', 8: 'bench', 9: 'bus_stop', 10: 'utility_pole', 11: 'motorcycle', 12: 'obstacle', 13: 'traffic light', 14: 'green', 15: 'elevator', 16: 'stair', 17: 'manhole', 18: 'dog', 19: 'trash', 20: 'red', 21: 'train_station'}
obb: None
orig_img: array([[[103, 140, 164],
        [ 95, 132, 155],
        [ 79, 126, 154],
        ...,
        [133, 134, 132],
        [133, 134, 132],
        [133, 134, 132]],

       [[103, 140, 164],
        [ 95, 132, 155],
        [ 79, 126, 154],
        ...,
        [133, 134, 132],
        [133, 134, 132],
        [133, 134, 132]],

       [[103, 140, 164],
        [ 95, 132, 155],
        [ 79, 126, 154],
        ...,
        [133, 134, 132],
        [133, 134, 132],
        [133, 134, 132]],

       ...,

       [[20

In [ ]:
# #iou start

# # iou_box 정의
# iou_box = [int(w/4), int(h/4), int(3*w/4), int(3*h/4)]
# iou_box_area = (iou_box[2] - iou_box[0]) * (iou_box[3] - iou_box[1])

# # 위험 감지 관련 임계값 설정
# iou_threshold = 0.8
# max_detections = 5  # 최대 시각화할 이미지 수
# detections = 0  # 현재까지 시각화한 이미지 수

# #iou end

# # #distance start

# #iou 박스 기준으로 하고 싶을때
# # iou_center_x = (iou_box[0] + iou_box[2]) / 2  # IoU 박스의 왼쪽과 오른쪽 x 좌표의 평균
# # iou_center_y = iou_box[3]  # IoU 박스의 하단 y 좌표
# # center_point = (int(iou_center_x), int(iou_center_y))

# center_point = (w/2,h-40)

# pixel_per_meter = 10

# txt_color, txt_background, bbox_clr = ((0, 0, 0), (255, 255, 255), (255, 0, 255))

# #distance end





# while True:
#     ret, im0 = cap.read()
#     if not ret:
#         print("Video frame is empty or video processing has been successfully completed.")
#         break
    


#     # iou 
#     results = model.track(im0, persist=True)
#     boxes = results[0].boxes.xyxy.cpu().numpy()

#     if results[0].boxes.id is not None:
#         track_ids = results[0].boxes.id.int().cpu().tolist()
    
#     detected_boxes = [(10, 20, 110, 220), (150, 160, 300, 460)]  # 예시 바운딩 박스

#     #IoU 박스 그리기
#     cv2.rectangle(im0, (iou_box[0], iou_box[1]), (iou_box[2], iou_box[3]), (255, 0, 0), 2)

#     for box, track_id in zip(boxes, track_ids):
#         # 현재 바운딩 박스의 영역 계산
#         current_box_area = (box[2] - box[0]) * (box[3] - box[1])
#         intersection_area, remaining_iou_area = calculate_iou_and_remaining_area(iou_box, box)
#          # remaining_iou_area이 남은 넓이임
    
#         # 남은 IoU 영역이 임계값 이하인 경우 경고 및 이미지 저장
#         if remaining_iou_area < iou_threshold * iou_box_area:
#             # 경고 메시지 출력 또는 시각적 표시
#             cv2.rectangle(im0, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 0, 255), 2)
#             x1, y1 = int((box[0] + box[2]) / 2), int((box[1] + box[3]) / 2)
            
#             # 이 부분이 중요: 각 객체에 대한 올바른 track_id를 사용
#             name = name_dict.get(track_id, "Unknown")  # track_id에 대응하는 이름을 가져옴


#             cv2.putText(im0, f"DANGER. {name} is ahead.{track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
#             cv2.waitKey(1)

#         # 위험 상황 시 이미지 저장
#         if save_counter < max_saves:
#             save_path = os.path.join(save_directory, f"danger_{save_counter}.jpg")
#             cv2.imwrite(save_path, im0)
#             print(f"Image saved to {save_path}")
#             save_counter += 1

            
        
        

#     #거리감지
#     annotator = Annotator(im0, line_width=2)
#     boxes = results[0].boxes.xyxy.cpu()
#     # 이전 프레임의 바운딩 박스 크기와 위치 정보 초기화
#     prev_box_area = None

#     prev_time = None
#     prev_x = None
#     prev_y = None

#     # 현재 프레임 처리 시작 시간
#     current_time = time.time()

#     # 현재 프레임의 바운딩 박스 크기 계산
#     if boxes.name == True:
#         current_box_area = (box[2] - box[0]) * (box[3] - box[1])
#     else:
#         pass

#     # 바운딩 박스 크기가 커지는지 확인
#     box_growing = prev_box_area is not None and current_box_area > prev_box_area

#     if results[0].boxes.id is not None:
#         track_ids = results[0].boxes.id.int().cpu().tolist()

#         # 초기 설정
#         prev_box_area = None
#         prev_time = None
#         prev_x = None
#         prev_y = None
#         speed = 0  # 초기 속도는 0으로 설정

#         # 현재 프레임 처리 시작 시간
#         current_time = time.time()

#         for box, track_id in zip(boxes, track_ids):


#             box_center_x = (box[0] + box[2]) / 2
#             center_x = int(center_point[0])
#             center_point_int = (int(center_point[0]), int(center_point[1]))

#             if box_center_x < center_x:
#                 right_top_x = int(box[2])  # 바운딩 박스 우측 x 좌표
#                 right_top_y = int(box[1])  # 바운딩 박스 상단 y 좌표
#                 cv2.line(im0, (right_top_x, right_top_y), center_point_int, (255, 0, 0), 2)
#             else:
#                 left_top_x = int(box[0])  # 바운딩 박스 왼쪽 x 좌표
#                 left_top_y = int(box[1])  # 바운딩 박스 상단 y 좌표
#                 cv2.line(im0, (left_top_x, left_top_y), center_point_int, (255, 0, 0), 2)

#             # 바운딩 박스 중심점과 크기 계산
#             x1, y1 = int((box[0] + box[2]) // 2), int((box[1] + box[3]) // 2)
#             current_box_area = (box[2] - box[0]) * (box[3] - box[1])
#             box_center_x = (box[0] + box[2]) / 2

#             # 바운딩 박스 크기 변화 확인
#             box_growing = prev_box_area is not None and current_box_area > prev_box_area

#             # 각도 및 거리 계산
#             angle = calculate_angle(x1, y1, center_point[0], center_point[1])
#             distance = math.sqrt((x1 - center_point[0])**2 + (y1 - center_point[1])**2) / pixel_per_meter

#             # 속도 계산
#             if prev_time is not None and prev_x is not None and prev_y is not None:
#                 time_diff = current_time - prev_time
#                 if time_diff > 0:  # 시간 차이가 0보다 클 때만 계산
#                     distance_moved = math.sqrt((x1 - prev_x)**2 + (y1 - prev_y)**2)
#                     speed = distance_moved / time_diff  # 속도 = 거리 / 시간
            
#             # 경고 메시지 조건 확인 및 표시
#             if 0 <= abs(angle) <= 30 and distance < 20:
#                 warning_message = "근처에 객체가 있어요"
#                 cv2.putText(im0, warning_message, (x1, y1 - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
#                 cv2.waitKey(1)

#                 if save_counter < max_saves:
#                     save_path = os.path.join(save_directory, f"danger_{save_counter}.jpg")
#                     cv2.imwrite(save_path, im0)
#                     print(f"Image saved to {save_path}")
#                     save_counter += 1

                

#             if speed > 25 and box_growing:
#                 warning_message = "객체가 다가오고 있습니다. 조심하세요!"
#                 cv2.putText(im0, warning_message, (x1, y1 - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
#                 cv2.waitKey(1)

#                 if save_counter < max_saves:
#                     save_path = os.path.join(save_directory, f"danger_{save_counter}.jpg")
#                     cv2.imwrite(save_path, im0)
#                     print(f"Image saved to {save_path}")
#                     save_counter += 1



#             # 결과 및 거리 정보 표시
#             cv2.putText(im0, f"Distance: {distance:.2f} m", (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 1.2, txt_color, 3)

#             # 프레임 정보 업데이트
#             prev_box_area = current_box_area
#             prev_time = current_time
#             prev_x = x1
#             prev_y = y1

#             # 결과 이미지 표시
#             cv2.imshow('Result', im0)
    
              
#             text_size, _ = cv2.getTextSize(f"Distance: {distance:.2f} m", cv2.FONT_HERSHEY_SIMPLEX,1.2, 3)
#             cv2.rectangle(im0, (x1, y1 - text_size[1] - 10),(x1 + text_size[0] + 10, y1), txt_background, -1)
#             cv2.putText(im0, f"Distance: {distance:.2f} m",(x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 1.2,txt_color, 3)

            



        
#         cv2.imshow('Processed Frame', im0)

#         # 'q' 키를 누르면 루프를 빠져나옵니다.
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break

# cap.release()
# cv2.destroyAllWindows()

